In [1]:
!unzip DCGRU_github.zip

Archive:  DCGRU_github.zip
  inflating: dcgru_cell_tf2.py       
   creating: lib/
  inflating: lib/matrix_calc.py      
   creating: synth_signal/
  inflating: synth_signal/gen_signal.py  
  inflating: synth_signal/road_network.py  
  inflating: signal.pickle           


# Import libraries

In [2]:
import numpy as np
import networkx as nx
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import backend as K
import matplotlib.pyplot as plt
import pickle

from synth_signal.gen_signal import generate_signal
from dcgru_cell_tf2 import DCGRUCell

# Load signal and adjacency matrix

In [7]:
signal = pickle.load(open('signal.pickle','rb'))
G_adj_mx = pickle.load(open('G_adj_mx.pickle','rb'))
num_nodes = G_adj_mx.shape[0]

# How to build a DCGRU layer

In [14]:
# define the dcgru cell
dcgru_cell = DCGRUCell(units=20,adj_mx=G_adj_mx, K_diffusion=2,
                       num_nodes=num_nodes,filter_type="random_walk")

The state size of the dcgru cell is a vector of dimension (units * num_nodes). In the example we have units = 20 and num_nodes = 24, hence the state size will be 20*24 = 480

In [15]:
# wrap the dcgru cell in a keras RNN layer
Dcgru_layer = keras.layers.RNN(dcgru_cell)

# Example of a sequential model

In [16]:
model_dcgru = keras.models.Sequential([
  keras.Input(shape=(None, num_nodes, 1)),  # Input dimensions: sequence length (None := arbitrary length)
  Dcgru_layer,                              #                   number of nodes in the graph
  keras.layers.Dense(num_nodes)             #                   signal dimensionality (1 in the example)
  ])

In [17]:
model_dcgru.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn_1 (RNN)                  (None, 480)               3840      
_________________________________________________________________
dense (Dense)                (None, 24)                11544     
Total params: 15,384
Trainable params: 15,384
Non-trainable params: 0
_________________________________________________________________


## Data format

In [19]:
# create X, y from signal
seq_len = 8
horizon = 4
X = []
y = []

for i in range(100,len(signal)-(seq_len+horizon)):
  X.append(signal[i:i+seq_len])
  y.append(signal[i+seq_len+horizon])
X = np.expand_dims(np.array(X), -1)   # expand dimensionality to match the correct input shape
y = np.array(y)

In [22]:
print(X.shape)
print(X.shape[0], ": number of input sequences")
print(X.shape[1], ": sequence length")
print(X.shape[2], ": number of nodhs")
print(X.shape[3], ": signal dimensionality")

(1888, 8, 24, 1)
1888 : number of input sequences
8 : sequence length
24 : number of nodhs
1 : signal dimensionality


In [24]:
print(y.shape)

(1888, 24)


## Model training

In [23]:
opt = keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)
model_dcgru.compile(loss="mse", optimizer=opt)
model_dcgru.fit(X, y, epochs=10, verbose=1)

Epoch 1/10
59/59 [==============================] - 2s 29ms/step - loss: 0.1051
Epoch 2/10
59/59 [==============================] - 2s 29ms/step - loss: 0.0121
Epoch 3/10
59/59 [==============================] - 2s 30ms/step - loss: 0.0094
Epoch 4/10
59/59 [==============================] - 2s 29ms/step - loss: 0.0073
Epoch 5/10
59/59 [==============================] - 2s 29ms/step - loss: 0.0058
Epoch 6/10
59/59 [==============================] - 2s 29ms/step - loss: 0.0050
Epoch 7/10
59/59 [==============================] - 2s 29ms/step - loss: 0.0047
Epoch 8/10
59/59 [==============================] - 2s 29ms/step - loss: 0.0046
Epoch 9/10
59/59 [==============================] - 2s 29ms/step - loss: 0.0045
Epoch 10/10
59/59 [==============================] - 2s 29ms/step - loss: 0.0044
